### Thesis Research Notebook <br>
### LDR Data Augmentation for Convolutional Neural Network Construction 

In [2]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()
assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

169017344/169001437 [==============================] - 20s 0us/step


array([[255, 255, 255],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [254, 254, 254],
       [252, 251, 250],
       [253, 246, 231],
       [218, 200, 179],
       [111,  90,  81],
       [163, 155, 155],
       [255, 255, 255],
       [255, 254, 254],
       [253, 252, 253],
       [253, 254, 253],
       [252, 254, 254],
       [247, 252, 253],
       [216, 227, 222],
       [198, 211, 194],
       [170, 176, 150],
       [161, 168, 130],
       [146, 154, 113]], dtype=uint8)